In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
from pprint import pprint
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

C:\Users\Shrey\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
os.getcwd()

'C:\\Users\\Shrey'

In [4]:
#Change directory to where dataset is
os.chdir("X:\\Datasets")

In [5]:
transaction_data=pd.read_excel('Fraud_Analytics_Dataset.xlsx',0)
customer_data=pd.read_excel('Fraud_Analytics_Dataset.xlsx',1)
sales_data=pd.read_excel('Fraud_Analytics_Dataset.xlsx',2)

In [6]:
display(transaction_data.head())
display(customer_data.head())
display(sales_data.head())

,Transaction_Id,First Time Customers.,Bigger than Average Orders.,Fast Shipping,Unusual Location.,Large quantity of Same Product .,Multiple Shipping Addresses.,Shipping and Billing Address are not the same.,Several Cards used from the Same Ip Address.,Payment Information typed with Capital letters.,Many Transactions in a short time period.
0,235360,No,No,No,No,No,No,No,No,No,No
1,234773,No,No,No,No,No,No,No,No,No,No
2,235242,No,No,No,No,No,No,No,No,No,No
3,235350,No,Yes,Yes,No,No,Yes,Yes,Yes,Yes,Yes
4,234914,No,No,No,No,No,No,No,No,No,No


,CUSTOMER_ID,LOYALTY_ID,MEMBERSHIP_TYPE__ID,MEMBERSHIP_TYPE,MEMBERSHIP_TYPE_SC,CUSTOMER_FLG,Customer_Type,customer_type_id,customer_type_sc,behavior_segment,...,BIRTH_MONTH,BIRTH_YEAR,BIRTH_QUARTER,PLACE_OF_BIRTH,COUNTRY_OF_BIRTH_SC,EVER_BANKRUPT_FLG,NO_TIMES_DELINQUENT_in_365_days,BLACKLISTED_FLG,NO_OF_DEPENDANTS,Record_created_By
0,76565665,LID877073,901.0,Gold,GLD,Y,Small/Medium Enterprise,1002.0,ENT,High Value,...,April,2004,2,Bellevue,US,N,1,N,3,\r
1,76565666,LID877029,NaN,NaN,NaN,Y,Individual,1003.0,INDV,Low Value,...,December,1984,4,Jackson,US,N,1,N,1,\r
2,76565667,LID876876,902.0,Silver,SLV,Y,Small/Medium Enterprise,1002.0,ENT,Medium Value,...,February,1999,1,Chattanooga,US,N,1,N,3,\r
3,76565668,LID876841,902.0,Silver,SLV,Y,Small/Medium Enterprise,1002.0,ENT,Low Value,...,December,1997,4,Worcester,US,N,0,N,1,\r
4,76565669,LID877001,902.0,Silver,SLV,Y,Individual,1003.0,INDV,Medium Value,...,January,1992,1,Los Angeles,US,N,1,N,1,\r


,Transaction_Id,Invoice_Id,Order_Id,Customer_Id,Mdm_Customer_Id,Loyalty_Id,Gender,Age,Membership_Id,Membership_Type,...,Cancel_Reason_Code,Cancel_Description,Return_Flag,Return_Dt,Return_Amt,Order_Returnpaid_Gross_Amt,Lastmodified_Dt,Sales_Rep_Name,Sales_Rep_Id,Fraud
0,235459,88203,88203,76565680,967122832,NaN,MALE,38,NaN,NaN,...,0,NaN,Y,2016-03-31,5622,4441,2016-10-14,Dave,S105\r,No
1,234676,88442,88442,76566015,978889718,NaN,MALE,36,NaN,NaN,...,0,NaN,N,2016-03-31,0,0,2016-05-24,NaN,\r,Yes
2,235246,87976,87976,76566552,914799361,LID876566,FEMALE,38,903.0,Bronze,...,0,NaN,N,2016-03-31,0,0,2016-08-04,NaN,\r,No
3,235283,88200,88200,76565868,921361924,NaN,FEMALE,34,NaN,NaN,...,0,NaN,N,2016-03-31,0,0,2016-10-11,Mark,S102\r,No
4,235296,87952,87952,76565700,958327875,LID876647,FEMALE,19,901.0,Gold,...,0,NaN,N,2016-03-31,0,0,2016-07-11,NaN,\r,No


In [7]:
'''
Joining all the 3 sheets
'''
#caller.join(other.set_index('key'), on='key')
f1=sales_data.join(transaction_data.set_index('Transaction_Id'),on='Transaction_Id')
original_data = f1.join(customer_data.set_index('CUSTOMER_ID'),on='Customer_Id')

In [8]:
#Preprocessing - > Wrangling + Cleaning

# Correcting column names for ease of usage
original_data.columns = original_data.columns.str.replace(' ','')
original_data.columns = original_data.columns.str.replace('.','')
        
original_data.head(10)

,Transaction_Id,Invoice_Id,Order_Id,Customer_Id,Mdm_Customer_Id,Loyalty_Id,Gender,Age,Membership_Id,Membership_Type,...,BIRTH_MONTH,BIRTH_YEAR,BIRTH_QUARTER,PLACE_OF_BIRTH,COUNTRY_OF_BIRTH_SC,EVER_BANKRUPT_FLG,NO_TIMES_DELINQUENT_in_365_days,BLACKLISTED_FLG,NO_OF_DEPENDANTS,Record_created_By
0,235459,88203,88203,76565680,967122832,NaN,MALE,38,NaN,NaN,...,December,1986,4,Norfolk,US,N,1,N,1,\r
1,234676,88442,88442,76566015,978889718,NaN,MALE,36,NaN,NaN,...,October,1999,4,Chula Vista,US,N,0,N,1,\r
2,235246,87976,87976,76566552,914799361,LID876566,FEMALE,38,903.0,Bronze,...,March,1990,1,Oakland,US,N,1,N,1,\r
3,235283,88200,88200,76565868,921361924,NaN,FEMALE,34,NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,0,NaN,0,\r
4,235296,87952,87952,76565700,958327875,LID876647,FEMALE,19,901.0,Gold,...,March,1985,1,Jackson,US,N,1,N,1,\r
5,234714,88472,88472,76565786,965982087,NaN,MALE,22,NaN,NaN,...,December,1977,4,Tampa,US,N,0,N,1,\r
6,234841,88736,88736,76566563,924634299,LID876577,MALE,36,903.0,Bronze,...,January,1984,1,Jersey City,US,N,1,N,1,\r
7,235101,87910,87910,76566223,954136284,LID876603,FEMALE,35,904.0,Platinum,...,December,1980,4,Irvine,US,N,1,N,2,\r
8,235115,88084,88084,76565784,975198853,LID876577,MALE,36,903.0,Bronze,...,September,1983,3,Hartford,US,N,0,N,3,\r
9,235276,88688,88688,76566561,962329895,LID876631,FEMALE,38,902.0,Silver,...,June,1995,2,Jacksonville,US,N,0,N,1,\r


In [9]:
#Please don't touch this piece of code, otherwise the destruction caused would be equivalent to World War 3
master_data = original_data

In [10]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Columns: 157 entries, Transaction_Id to Record_created_By
dtypes: datetime64[ns](4), float64(13), int64(39), object(101)
memory usage: 1.1+ MB


---
Through a basic Excel Sheet analysis, we have are seeing a lot of redundant variables as well as duplicate ones. 

1. Columns representing unique values such as Transaction ID's or Customer ID's can be removed directly.

2. Multiple variables such as Loyalty ID, Membership related Info, etc. are representing the same information, leading to **Multicollinearity in the data**. Such variables can be romoved or clubbed together to create a single informative column.

3. Some variables, which are not relevant within the aim of this project, such as Country, can be removed, as they contains only a single value. If and when this data is extrapolated for other countries, it may prove to be useful.

4. Fields with large number of missing values, as we can't assume the values in such cases for better chances of accuracy.

We will not be using such columns, and a single list of columns being used for model building will be informed.

---

In [11]:
# Initial features list for feature selection at model building stages
feature_list = ['FastShipping','credit_risk_score','age_range','Interest_2','INFLUENCE',
 'Interests_1', 'INCOME_RANGE',
 'LOYALITY_FLG', 'NO_TIMES_DELINQUENT_in_365_days',
 'City_Sc', 'credit_risk_rating',
 'CUSTOMER_STATUS_SC',
 'fbck_user_FLG', 'NO_OF_DEPENDANTS',
 'No_Of_Items', 'Shipment_Flg',
 'Order_Item_Paidprice_Gross_Amt_',
 'REGION', 'fbck_following_CNT',
 'Order_Item_Listprice_Gross_Amt_','twt_follower_CNT','BiggerthanAverageOrders',
 'No_Of_Items_After_Return',
 'Return_Flag','MultipleShippingAddresses',
 'SeveralCardsusedfromtheSameIpAddress',
 'UnusualLocation','ShippingandBillingAddressarenotthesame',
 'Order_Returnpaid_Gross_Amt',
 'Return_Amt','PaymentInformationtypedwithCapitalletters',
 'Customer_Type','Lastmodified_Dt',
 'FirstTimeCustomers','Discount_Percentage',
 'customer_type_sc','Quantity_Returned',
 'Discount_Amt','Cancel_Date',
 'ManyTransactionsinashorttimeperiod',
 'Mode_Of_Payment','Return_Dt']
target = 'Fraud'

In [12]:
# Segregating Target Data and Feature Set
target = 'Fraud'
label_data = master_data[target].astype('category').cat.codes  # Converting Fraud to binary categorical variables [0,1]
master_data.drop('Fraud', axis = 1)

master_data = master_data[feature_list]
master_data.info()

0    0
1    1
2    0
3    0
4    0
dtype: int8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 42 columns):
FastShipping                                 947 non-null object
credit_risk_score                            947 non-null int64
age_range                                    921 non-null object
Interest_2                                   459 non-null object
INFLUENCE                                    921 non-null object
Interests_1                                  460 non-null object
INCOME_RANGE                                 921 non-null object
LOYALITY_FLG                                 921 non-null object
NO_TIMES_DELINQUENT_in_365_days              947 non-null int64
City_Sc                                      947 non-null object
credit_risk_rating                           921 non-null object
CUSTOMER_STATUS_SC                           921 non-null object
fbck_user_FLG                                921 non-null object
NO_OF

In [13]:
# Finding columns with datatypes which classifiers will not recognise such as object and datetime
obj_type_variables = [column for column in master_data.columns if master_data[column].dtype in ['object', 'datetime64[ns]']]
print(obj_type_variables)

['FastShipping', 'age_range', 'Interest_2', 'INFLUENCE', 'Interests_1', 'INCOME_RANGE', 'LOYALITY_FLG', 'City_Sc', 'credit_risk_rating', 'CUSTOMER_STATUS_SC', 'fbck_user_FLG', 'Shipment_Flg', 'REGION', 'BiggerthanAverageOrders', 'Return_Flag', 'MultipleShippingAddresses', 'SeveralCardsusedfromtheSameIpAddress', 'UnusualLocation', 'ShippingandBillingAddressarenotthesame', 'PaymentInformationtypedwithCapitalletters', 'Customer_Type', 'Lastmodified_Dt', 'FirstTimeCustomers', 'customer_type_sc', 'Cancel_Date', 'ManyTransactionsinashorttimeperiod', 'Mode_Of_Payment', 'Return_Dt']


In [14]:
# Making object type data into categorical variables, for creating categorical codes
for column in obj_type_variables:
    master_data[column] = master_data[column].astype('category')

C:\Users\Shrey\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 42 columns):
FastShipping                                 947 non-null category
credit_risk_score                            947 non-null int64
age_range                                    921 non-null category
Interest_2                                   459 non-null category
INFLUENCE                                    921 non-null category
Interests_1                                  460 non-null category
INCOME_RANGE                                 921 non-null category
LOYALITY_FLG                                 921 non-null category
NO_TIMES_DELINQUENT_in_365_days              947 non-null int64
City_Sc                                      947 non-null category
credit_risk_rating                           921 non-null category
CUSTOMER_STATUS_SC                           921 non-null category
fbck_user_FLG                                921 non-null category
NO_OF_DEPENDANTS              

In [16]:
# Converting categorical into corresponding numerical values
master_data[obj_type_variables] = master_data[obj_type_variables].apply(lambda column: column.cat.codes)

C:\Users\Shrey\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [17]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 42 columns):
FastShipping                                 947 non-null int8
credit_risk_score                            947 non-null int64
age_range                                    947 non-null int8
Interest_2                                   947 non-null int8
INFLUENCE                                    947 non-null int8
Interests_1                                  947 non-null int8
INCOME_RANGE                                 947 non-null int8
LOYALITY_FLG                                 947 non-null int8
NO_TIMES_DELINQUENT_in_365_days              947 non-null int64
City_Sc                                      947 non-null int8
credit_risk_rating                           947 non-null int8
CUSTOMER_STATUS_SC                           947 non-null int8
fbck_user_FLG                                947 non-null int8
NO_OF_DEPENDANTS                             947 non-null int64
No_Of_Item

In [230]:
#list of columns with null values 
list_of_clm_null_value= master_data.columns[master_data.isna().any()].tolist()
print(list_of_clm_null_value)

null_columns=master_data.columns[master_data.isnull().any()]
count = master_data[null_columns].isnull().sum()
print(count)

[]
Series([], dtype: float64)


In [20]:
# replacing the NaN value with -1 after EDA
for i in list_of_clm_null_value:
    master_data[i].fillna(-1, inplace = True) 

In [21]:
# Converting numerical based categories into category dtype
for column in obj_type_variables:
    master_data[column] = master_data[column].astype('category')
master_data.info()

C:\Users\Shrey\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 42 columns):
FastShipping                                 947 non-null category
credit_risk_score                            947 non-null int64
age_range                                    947 non-null category
Interest_2                                   947 non-null category
INFLUENCE                                    947 non-null category
Interests_1                                  947 non-null category
INCOME_RANGE                                 947 non-null category
LOYALITY_FLG                                 947 non-null category
NO_TIMES_DELINQUENT_in_365_days              947 non-null int64
City_Sc                                      947 non-null category
credit_risk_rating                           947 non-null category
CUSTOMER_STATUS_SC                           947 non-null category
fbck_user_FLG                                947 non-null category
NO_OF_DEPENDANTS              

---
Model Building
---

We will be testing various models for this Binary Classification problem.

Our approach would be to first create baseline untuned models on a pre-decided ratio of training and testing datasets, and after that, through various iterations, arrive at the tuned model which gives a balanced result, with a >=85% accuracy.

Along with accuracy, our metrics for evaluation of the model would be Precision, Recall and Confusion matrix.

---

In [221]:
#Stratified Shuffle Split by 70:30 ratio + SelectKBest
retail_data = master_data
skb = SelectKBest(k=8)
retail_data = skb.fit_transform(retail_data, label_data)
#features_train, features_test, labels_train, labels_test = train_test_split(retail_data, label_data, test_size=0.3, random_state=42)

splitter = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
for train_index, test_index in splitter.split(retail_data, label_data):
        features_train, features_test = retail_data[train_index], retail_data[test_index]
        labels_train, labels_test = label_data[train_index], label_data[test_index]

In [222]:
# Getting feature scores from SelectKBest after fitting
unsorted_list = zip(feature_list, skb.scores_)

sorted_features = sorted(unsorted_list, key=lambda x: x[1], reverse=True)
print(len(sorted_features))
print("Feature Scores:\n")
pprint(sorted_features)

42
Feature Scores:

[('FastShipping', 8.100001301014169),
 ('BiggerthanAverageOrders', 5.745887795284878),
 ('No_Of_Items_After_Return', 4.916972422311143),
 ('credit_risk_score', 4.381886936924487),
 ('No_Of_Items', 4.190314173770475),
 ('age_range', 3.804572951937072),
 ('Interest_2', 3.7669951329272315),
 ('Return_Flag', 3.4152996363954338),
 ('Shipment_Flg', 3.3090168010633767),
 ('MultipleShippingAddresses', 3.125091243167083),
 ('SeveralCardsusedfromtheSameIpAddress', 2.8029111724079567),
 ('UnusualLocation', 2.424618839948198),
 ('ShippingandBillingAddressarenotthesame', 2.1373884646910644),
 ('PaymentInformationtypedwithCapitalletters', 1.766565168678694),
 ('INFLUENCE', 1.6322174530054272),
 ('Interests_1', 1.5553705001377034),
 ('Customer_Type', 1.3879597390041023),
 ('INCOME_RANGE', 1.3067636521158699),
 ('Lastmodified_Dt', 1.2953773334508918),
 ('Return_Amt', 1.2294380875470237),
 ('FirstTimeCustomers', 1.1803379328321633),
 ('Discount_Percentage', 1.1536995086191966),
 ('O

In [224]:
# Trying untuned, baseline models on 70:30 train-test dataset
gnb = GaussianNB()
y_pred_gnb = gnb.fit(features_train, labels_train).predict(features_test)

DT = DecisionTreeClassifier()
y_pred_DT = DT.fit(features_train, labels_train).predict(features_test)

ada = AdaBoostClassifier()
y_pred_ada = ada.fit(features_train, labels_train).predict(features_test)

logr = LogisticRegression()
y_pred_logr = logr.fit(features_train, labels_train).predict(features_test)

In [225]:
# Accuracy Metrics
print("GNB:", accuracy_score(labels_test, y_pred_gnb))
print("DT:", accuracy_score(labels_test, y_pred_DT))
print("ADA:", accuracy_score(labels_test, y_pred_ada))
print("Logr:", accuracy_score(labels_test, y_pred_logr))

GNB: 0.8070175438596491
DT: 0.7859649122807018
ADA: 0.8526315789473684
Logr: 0.8666666666666667


In [231]:
# Precision, Recall, and actual prediction numbers
print("GNB:", classification_report(labels_test, y_pred_gnb))
print("DT:", classification_report(labels_test, y_pred_DT))
print("ADA:", classification_report(labels_test, y_pred_ada))
print("Logr:", classification_report(labels_test, y_pred_logr))

GNB:              precision    recall  f1-score   support

          0       0.88      0.90      0.89       248
          1       0.20      0.16      0.18        37

avg / total       0.79      0.81      0.80       285

DT:              precision    recall  f1-score   support

          0       0.87      0.88      0.88       248
          1       0.15      0.14      0.14        37

avg / total       0.78      0.79      0.78       285

ADA:              precision    recall  f1-score   support

          0       0.87      0.98      0.92       248
          1       0.00      0.00      0.00        37

avg / total       0.76      0.85      0.80       285

Logr:              precision    recall  f1-score   support

          0       0.87      1.00      0.93       248
          1       0.00      0.00      0.00        37

avg / total       0.76      0.87      0.81       285



---
### Tuning the classifiers

We will be using a pipeline, with a MinMax Scaler at stage 1 for int/float values, and the classifier at stage 2. This pipeline will be fed to GridSearchCV along with the classifier instance, as well as parameter list for that particular classifier.

Through previously mentioned metrics, we will be determining the best tuned classifier here.

---

In [227]:
def tune_NB() :
    print("------------------ Using Naive Bayes --------------------")
    nb_clf = GaussianNB()
    param_grid = {}             # No parameters for tuning

    return nb_clf, param_grid

# Decision Tree Classifier
def tune_DT():
    print("------------------ Using Decision Trees --------------------")
    clf = DecisionTreeClassifier()
    param_grid = {
        'clf__criterion': ['entropy', 'gini'],
        'clf__splitter': ['best', 'random'],
        'clf__random_state': [42],
        'clf__min_samples_split': [2, 4, 5, 6, 7, 8]
    }

    return clf, param_grid

# AdaBoost Classifier
def tune_ADB():
    print("------------------ Using AdaBoost Ensemble --------------------")
    clf = AdaBoostClassifier()
    param_grid = {
        'clf__algorithm' : ['SAMME', 'SAMME.R'],
        'clf__learning_rate': [1, 2],
        'clf__random_state': [42],
        'clf__n_estimators' : [20, 50, 65, 80, 100]
    }

    return clf, param_grid

# Logistic Regression Classifier
def tune_LogR():
    print("------------------ Using Logistic Regression --------------------")
    clf = LogisticRegression()
    param_grid = {
        'clf__max_iter' : [100, 500, 1000],
        'clf__penalty': ['l1', 'l2'],
        'clf__tol': [0.0001, 0.0005, 0.001, 0.005],
        'clf__C' : [1.0, 1.5, 0.5, 2.0]
    }

    return clf, param_grid

# Random Forest Classifier
def tune_RandomF():
    print("------------------ Using Random Forest --------------------")
    clf = RandomForestClassifier()
    param_grid = {
        'clf__criterion': ['entropy', 'gini'],
        'clf__min_samples_split': [2, 4, 5, 6, 7, 8, 9, 10],
        'clf__random_state': [42],
        'clf__n_estimators' : [20, 40, 60, 80, 100, 200],
        'clf__max_features' : ['auto', 'log2', None] 
    }

    return clf, param_grid

In [228]:
 # Create pipeline
clf, params = tune_DT()
scale = MinMaxScaler()
estimators = [('scale', scale), ('clf', clf)]
pipe = Pipeline(estimators)

# Create GridSearchCV Instance
grid = GridSearchCV(pipe, params)
grid.fit(features_train, labels_train)

# Final classifier
clf = grid.best_estimator_

print('\n=> Chosen parameters :')
print(grid.best_params_)

------------------ Using Decision Trees --------------------

=> Chosen parameters :
{'clf__criterion': 'gini', 'clf__min_samples_split': 8, 'clf__random_state': 42, 'clf__splitter': 'random'}


In [229]:
# Metrics
predictions = clf.predict(features_test)
print("Accuracy Score:", accuracy_score(labels_test, predictions))
print("Classification Report:\n", classification_report(labels_test, predictions))
tn, fp, fn, tp = confusion_matrix(labels_test, predictions).ravel()
print("Confusion Matrix:\n", tn, fp, fn, tp)

Accuracy Score: 0.856140350877193
Classification Report:
              precision    recall  f1-score   support

          0       0.88      0.96      0.92       248
          1       0.36      0.14      0.20        37

avg / total       0.81      0.86      0.83       285

Confusion Matrix:
 239 9 32 5


***Hence, after multiple iterations, we are achieving a respectable accuracy of 85.61% using a tuned Decision Tree Classifier. The dataset is split in 70-30 ration, with top 8 features being used for classification from SelectKBest.***

***We are receiving an average Precision Score of 0.81, and recall score of 0.86.***